In [1]:
import tensorflow as tf
import numpy as np
import random
import os
import matplotlib.pyplot as plt

In [2]:
trainlist, testlist = [], []
with open('train.txt') as f:
    for line in f:
        tmp = line.strip().split()
        trainlist.append([tmp[0], tmp[1]])
        
with open('test.txt') as f:
    for line in f:
        tmp = line.strip().split()
        testlist.append([tmp[0], tmp[1]])

In [3]:
IMG_H = 100
IMG_W = 100
IMG_C = 3

def readimg(path):
    img = plt.imread(path)
    return img

def batch(path, batch_size):
    img, label, paths = [], [], []
    for i in range(batch_size):
        img.append(readimg(path[0][0]))
        label.append(int(path[0][1]))
        path.append(path.pop(0))
        
    return img, label

In [14]:
num_class = 3 

with tf.Graph().as_default() as g:
    X = tf.placeholder(tf.float32, [None, IMG_H, IMG_W, IMG_C])
    Y = tf.placeholder(tf.int32, [None])
    
    with tf.variable_scope('CNN'):
        net = tf.layers.conv2d(X, 20, 3, (2, 2), padding='same', activation=tf.nn.relu)
        net = tf.layers.max_pooling2d(net, 2, 2)
        net = tf.layers.conv2d(net, 40, 3, (2, 2), padding='same', activation=tf.nn.relu)
        net = tf.layers.flatten(net)
     
        out = tf.layers.dense(net, num_class)
        
    with tf.variable_scope('Loss'):
        loss = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(labels= Y, logits=out))
        
    train = tf.train.AdamOptimizer(1e-3).minimize(loss)
    saver = tf.train.Saver()


In [18]:
np.sum([np.product(var.shape) for var in g.get_collection('trainable_variables')]).value

28083

In [16]:
batch_size = 1461
with tf.Session(graph=g) as sess:
    sess.run(tf.global_variables_initializer())
    for i in range(30):
        batch_data, batch_label = batch(trainlist, batch_size)
        _, l = sess.run([train, loss], feed_dict = {X: batch_data, Y: batch_label})
        print(i, l)
        
    saver.save(sess, 'logs/model.ckpt', global_step = i+1)

0 21.442257
1 37.37255
2 77.171
3 51.91084
4 13.559056
5 3.0863907
6 9.855622
7 14.757939
8 15.345281
9 13.067308
10 8.83835
11 3.841644
12 0.54752165
13 0.45238113
14 3.8108256
15 3.8439145
16 1.7248763
17 0.13583045
18 0.0013055985
19 0.031298477
20 0.10130094
21 0.22030635
22 0.2875862
23 0.2352724
24 0.12951691
25 0.06642132
26 0.016250532
27 0.0002915832
28 0.010168281
29 0.10443287


In [17]:
acc = 0
with tf.Session(graph=g) as sess:
    sess.run(tf.global_variables_initializer())
    checkpoint = tf.train.latest_checkpoint('logs')
    if checkpoint:
        saver.restore(sess, checkpoint)
    for i in range(len(testlist)):
        batch_data, batch_label = batch(testlist, 1)
        logit = sess.run(out, feed_dict = {X:batch_data})
        if np.argmax(logit[0]) == batch_label[0]:
            acc += 1
        else:
            print(logit[0], batch_label[0])
            
    print(acc/len(testlist))

INFO:tensorflow:Restoring parameters from logs\model.ckpt-30
[13.132436 -9.103183 11.965624] 2
[11.102251  -2.9212844 10.0971365] 2
0.9959183673469387
